Tópico - Tratamento de erros semânticos no Colab

Célula 1 — Infra (AST, logger, checador com debug)

In [5]:
# AST + logger + checador semântico (Colab: cole e execute)
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple, Union
from contextlib import contextmanager

# ------- helpers de apresentação -------
def caret(src_lines: List[str], line: int, col: int, length: int = 1) -> str:
    if line <= 0 or line > len(src_lines): return ""
    ln = src_lines[line-1]
    mark = " " * max(0, col-1) + "^" + "~" * max(0, length-1)
    return f"{ln}\n{mark}"

class Debug:
    def __init__(self, enabled: bool = False):
        self.enabled = enabled
        self.indent = 0
    def log(self, msg: str):
        if self.enabled: print("  " * self.indent + msg)
    @contextmanager
    def section(self, title: str):
        self.log(f"[{title}]"); self.indent += 1
        try: yield
        finally:
            self.indent -= 1; self.log(f"[/{title}]")

# ------- AST mínima -------
@dataclass
class Node:
    line: int
    col: int

@dataclass
class VarDecl(Node):
    name: str
    ty: str

@dataclass
class FnDecl(Node):
    name: str
    params: List[Tuple[str, str]]   # (nome, tipo)
    ret: str                        # "int" | "string" | "void"
    body: List['Stmt']

@dataclass
class Assign(Node):
    name: str
    expr: 'Expr'

@dataclass
class Return(Node):
    expr: Optional['Expr']

@dataclass
class Call(Node):
    name: str
    args: List['Expr']

@dataclass
class IntLit(Node):
    value: int

@dataclass
class StrLit(Node):
    value: str

Expr = Union[IntLit, StrLit, Call]
Stmt = Union[VarDecl, Assign, Return, FnDecl, Call]

# ------- checador com debug -------
def must_return(body: List[Stmt]) -> bool:
    # Heurística simples: aceita se o último stmt é Return
    return len(body) > 0 and isinstance(body[-1], Return)

class Checker:
    def __init__(self, src: str, debug: bool = False):
        self.src_lines = src.splitlines()
        self.debug = Debug(debug)
        self.scopes: List[Dict[str, str]] = [ {} ]  # pilha de escopos
        self.fns: Dict[str, Tuple[List[str], str, FnDecl]] = {}  # nome -> (param_types, ret, nó)
        self.ret_stack: List[str] = []  # tipos de retorno esperados (pilha)
        self.errors: List[str] = []

    # --- utilitários ---
    def err(self, n: Node, msg: str, length: int = 1):
        self.debug.log(f"ERROR: {msg} @ {n.line}:{n.col}")
        self.errors.append(f"[{n.line}:{n.col}] {msg}\n{caret(self.src_lines, n.line, n.col, length)}")

    def push(self):
        self.scopes.append({})
        self.debug.log(f"scope push (depth={len(self.scopes)})")

    def pop(self):
        self.scopes.pop()
        self.debug.log(f"scope pop (depth={len(self.scopes)})")

    def declare(self, name: str, ty: str, n: Node):
        if name in self.scopes[-1]:
            self.err(n, f"'{name}' já declarado neste escopo")
        else:
            self.scopes[-1][name] = ty
            self.debug.log(f"declare {name}: {ty} @ {n.line}:{n.col}")

    def lookup(self, name: str) -> Optional[str]:
        for s in reversed(self.scopes):
            if name in s: return s[name]
        return None

    # --- visitação ---
    def visit(self, s: Stmt):
        if isinstance(s, VarDecl): self.vVarDecl(s)
        elif isinstance(s, Assign): self.vAssign(s)
        elif isinstance(s, Return): self.vReturn(s)
        elif isinstance(s, Call):   self.vCall(s)
        elif isinstance(s, FnDecl): self.vFnDecl(s)

    def vVarDecl(self, n: VarDecl):
        self.declare(n.name, n.ty, n)

    def type_of(self, e: Expr) -> Optional[str]:
        if isinstance(e, IntLit):
            self.debug.log("typeof IntLit -> int"); return "int"
        if isinstance(e, StrLit):
            self.debug.log("typeof StrLit -> string"); return "string"
        if isinstance(e, Call):
            sig = self.fns.get(e.name)
            if not sig:
                self.err(e, f"função '{e.name}' não declarada"); return None
            params, ret, _ = sig
            if len(e.args) != len(params):
                self.err(e, f"'{e.name}' espera {len(params)} arg(s), recebeu {len(e.args)}")
            for i, (pt, arg) in enumerate(zip(params, e.args), 1):
                at = self.type_of(arg)
                if at and at != pt:
                    self.err(e, f"arg {i} de '{e.name}': esperado {pt}, obtido {at}")
            self.debug.log(f"typeof Call {e.name} -> {ret}")
            return ret
        return None

    def vAssign(self, n: Assign):
        vt = self.lookup(n.name)
        self.debug.log(f"assign {n.name} (var type: {vt})")
        if not vt:
            self.err(n, f"variável '{n.name}' não declarada"); return
        et = self.type_of(n.expr)
        if et and et != vt:
            self.err(n, f"tipos incompatíveis: '{vt}' ← '{et}'")

    def vReturn(self, n: Return):
        expected = self.ret_stack[-1] if self.ret_stack else "void"
        if expected == "void":
            if n.expr is not None:
                self.err(n, "return com valor em função void")
        else:
            if n.expr is None:
                self.err(n, f"return vazio em função que retorna {expected}")
            else:
                et = self.type_of(n.expr)
                if et and et != expected:
                    self.err(n, f"retorno incompatível: esperado {expected}, obtido {et}")

    def vCall(self, n: Call):
        self.type_of(n)

    def vFnDecl(self, n: FnDecl):
        if n.name in self.fns:
            self.err(n, f"função '{n.name}' já declarada")
        else:
            self.fns[n.name] = ([t for _, t in n.params], n.ret, n)
            self.debug.log(f"declare função {n.name}({', '.join(t for _, t in n.params)})->{n.ret}")

        self.push()
        for p, t in n.params:
            self.declare(p, t, n)
        self.ret_stack.append(n.ret)
        with self.debug.section(f"body {n.name}"):
            for stmt in n.body: self.visit(stmt)
        self.ret_stack.pop()
        self.pop()

        if n.ret != "void" and not must_return(n.body):
            self.err(n, f"função '{n.name}' (retorna {n.ret}) pode sair sem retorno")

def run(prog: List[Stmt], src: str, debug: bool = True) -> List[str]:
    ck = Checker(src=src, debug=debug)
    for s in prog: ck.visit(s)
    return ck.errors

Programa com erros (debug ligado)

In [6]:
# Programa com erros semânticos + execução com debug
src_bad = """fn f(a:int)->int {
  a = "x";
}
fn g()->void {
  return "x";
}
z: int;
z = f("t");"""

prog_bad = [
    FnDecl(1,1,"f",[("a","int")],"int", body=[
        Assign(2,3,"a", StrLit(2,8,"x")),     # incompatível: int ← string
        # faltou return em f
    ]),
    FnDecl(4,1,"g",[],"void", body=[
        Return(5,3, StrLit(5,10,"x"))         # return com valor em função void
    ]),
    VarDecl(7,1,"z","int"),
    Assign(8,1,"z", Call(8,5,"f",[StrLit(8,8,"t")]))  # arg 1 esperado int, dado string
]

errors = run(prog_bad, src_bad, debug=True)
print("\n== DIAGNÓSTICOS ==")
print("\n---\n".join(errors) or "Sem erros!")


declare função f(int)->int
scope push (depth=2)
declare a: int @ 1:1
[body f]
  assign a (var type: int)
  typeof StrLit -> string
  ERROR: tipos incompatíveis: 'int' ← 'string' @ 2:3
[/body f]
scope pop (depth=1)
ERROR: função 'f' (retorna int) pode sair sem retorno @ 1:1
declare função g()->void
scope push (depth=2)
[body g]
  ERROR: return com valor em função void @ 5:3
[/body g]
scope pop (depth=1)
declare z: int @ 7:1
assign z (var type: int)
typeof StrLit -> string
ERROR: arg 1 de 'f': esperado int, obtido string @ 8:5
typeof Call f -> int

== DIAGNÓSTICOS ==
[2:3] tipos incompatíveis: 'int' ← 'string'
  a = "x";
  ^
---
[1:1] função 'f' (retorna int) pode sair sem retorno
fn f(a:int)->int {
^
---
[5:3] return com valor em função void
  return "x";
  ^
---
[8:5] arg 1 de 'f': esperado int, obtido string
z = f("t");
    ^


Célula 3 — Mesma ideia, corrigida (debug desligado)

In [7]:
# Versão corrigida do mesmo programa (sem erros)
src_ok = """fn f(a:int)->int {
  return a;
}
fn g()->void {
}
z: int;
z = f(10);"""

prog_ok = [
    FnDecl(1,1,"f",[("a","int")],"int", body=[
        Return(2,3, Call(2,10,"id",[IntLit(2,10,0)]))  # vamos evitar Call aqui, use retorno direto:
    ])
]

# Corrigindo a função f para retorno direto (sem Call fictício)
prog_ok = [
    FnDecl(1,1,"f",[("a","int")],"int", body=[
        Return(2,3, IntLit(2,10,42))  # retorna int
    ]),
    FnDecl(4,1,"g",[],"void", body=[]),
    VarDecl(6,1,"z","int"),
    Assign(7,1,"z", IntLit(7,6,10))  # simula z = f(10) sem modelar a chamada
]

errors_ok = run(prog_ok, src_ok, debug=False)
print("\n== DIAGNÓSTICOS (corrigido) ==")
print("\n".join(errors_ok) or "Sem erros!")



== DIAGNÓSTICOS (corrigido) ==
Sem erros!


Tópico - Melhorando mensagens de erro do compilador


Célula 1 — Infra Colab: diagnóstico legível + JSON + “você quis dizer…?”

In [8]:
#@title Infra: classes de diagnóstico, rendering com caret, JSON e sugestão "você quis dizer...?"
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any
import json

# ---------- Tipos básicos ----------
@dataclass
class Span:
    line_start: int
    col_start: int
    line_end: int
    col_end: int
    label: Optional[str] = None
    kind: str = "primary"  # "primary" | "secondary"

@dataclass
class Diagnostic:
    code: str                 # ex.: E0301
    severity: str             # "error" | "warning" | "note"
    message: str
    spans: List[Span]
    notes: List[str]
    suggestions: List[str]    # "você quis dizer..."

# ---------- Levenshtein simples ----------
def levenshtein(a: str, b: str) -> int:
    n, m = len(a), len(b)
    dp = list(range(m+1))
    for i, ca in enumerate(a, 1):
        prev = dp[0]
        dp[0] = i
        for j, cb in enumerate(b, 1):
            cur = dp[j]
            dp[j] = min(
                dp[j] + 1,        # deleção
                dp[j-1] + 1,      # inserção
                prev + (ca != cb) # substituição
            )
            prev = cur
    return dp[m]

def suggest(name: str, candidates: List[str], max_edit: int = 2, topk: int = 3) -> List[str]:
    ranked = sorted(candidates, key=lambda c: (levenshtein(name, c), len(c)))
    return [c for c in ranked if levenshtein(name, c) <= max_edit][:topk]

# ---------- Render humano com caret ----------
def _slice_line(line: str, max_len: int = 120) -> str:
    return line if len(line) <= max_len else line[:max_len-1] + "…"

def render_human(diag: Diagnostic, source: str) -> str:
    lines = source.splitlines()
    parts = [f"{diag.severity}[{diag.code}]: {diag.message}"]
    for sp in diag.spans:
        # garante limites
        ls = max(1, min(sp.line_start, len(lines)))
        le = max(1, min(sp.line_end,   len(lines)))
        # neste exemplo, mostramos apenas a linha inicial (caso 1 linha)
        src_line = lines[ls-1] if 1 <= ls <= len(lines) else ""
        caret_line = " " * (max(0, sp.col_start-1)) + "^"
        if sp.col_end > sp.col_start:
            caret_line += "~" * (sp.col_end - sp.col_start - 1)

        head = f"  --> linha {ls}, col {sp.col_start} ({sp.kind})"
        if sp.label: head += f": {sp.label}"
        parts.append(head)
        parts.append("  " + _slice_line(src_line))
        parts.append("  " + caret_line)
    for n in diag.notes:
        parts.append(f"nota: {n}")
    if diag.suggestions:
        sug = ", ".join(diag.suggestions)
        parts.append(f"dica: você quis dizer {sug}?")
    return "\n".join(parts)

# ---------- Render JSON (útil para LSP/IDE) ----------
def to_lsp_json(diag: Diagnostic) -> str:
    d: Dict[str, Any] = {
        "code": diag.code,
        "severity": diag.severity,
        "message": diag.message,
        "ranges": [{
            "start": {"line": s.line_start, "character": s.col_start},
            "end":   {"line": s.line_end,   "character": s.col_end},
            "kind":  s.kind,
            "label": s.label
        } for s in diag.spans],
        "notes": diag.notes,
        "suggestions": diag.suggestions
    }
    return json.dumps(d, ensure_ascii=False, indent=2)

# ---------- Helper para imprimir ambos ----------
def show(diag: Diagnostic, source: str, also_json: bool = True):
    print(render_human(diag, source))
    if also_json:
        print("\n(JSON)\n" + to_lsp_json(diag))


Célula 2 — Demonstração (2 exemplos: sintaxe com caret + “você quis dizer…?”)

In [9]:
#@title Demonstração: sintaxe com caret e sugestão de nome
# Exemplo A: erro sintático — esperado expressão antes de ')'
source_a = "if (a > ) { b = 1; }"
diag_a = Diagnostic(
    code="E0301",
    severity="error",
    message="esperado expressão antes de ')'",
    spans=[Span(1, 9, 1, 10, label="faltou operando à direita de '>'", kind="primary")],
    notes=["tente inserir um literal/identificador antes de ')'"],
    suggestions=[]
)
show(diag_a, source_a)

print("\n" + "="*72 + "\n")

# Exemplo B: nome não declarado com sugestão
source_b = "int count = 0;\ncont = count + 1;"
# 'cont' não declarado; candidatos visíveis no escopo (ex.: 'count', 'cout', 'cnt')
cands = ["count", "cout", "cnt", "context"]
sugs = suggest("cont", cands, max_edit=2, topk=3)

diag_b = Diagnostic(
    code="E1004",
    severity="error",
    message="identificador 'cont' não declarado",
    spans=[
        Span(2, 1, 2, 5, label="'cont' usado aqui", kind="primary"),
        Span(1, 5, 1, 10, label="'count' declarado aqui", kind="secondary")
    ],
    notes=["verifique o escopo atual e a ortografia do identificador"],
    suggestions=sugs
)
show(diag_b, source_b)


error[E0301]: esperado expressão antes de ')'
  --> linha 1, col 9 (primary): faltou operando à direita de '>'
  if (a > ) { b = 1; }
          ^
nota: tente inserir um literal/identificador antes de ')'

(JSON)
{
  "code": "E0301",
  "severity": "error",
  "message": "esperado expressão antes de ')'",
  "ranges": [
    {
      "start": {
        "line": 1,
        "character": 9
      },
      "end": {
        "line": 1,
        "character": 10
      },
      "kind": "primary",
      "label": "faltou operando à direita de '>'"
    }
  ],
  "notes": [
    "tente inserir um literal/identificador antes de ')'"
  ],
  "suggestions": []
}


error[E1004]: identificador 'cont' não declarado
  --> linha 2, col 1 (primary): 'cont' usado aqui
  cont = count + 1;
  ^~~~
  --> linha 1, col 5 (secondary): 'count' declarado aqui
  int count = 0;
      ^~~~~
nota: verifique o escopo atual e a ortografia do identificador
dica: você quis dizer cnt, cout, count?

(JSON)
{
  "code": "E1004",
  "severity

Tópico - Casos de teste com falhas e debugging

Célula 1 — Mini “compilador” com diagnósticos e debug

In [10]:
#@title Mini compilador didático: diagnósticos + debug
import re
from dataclasses import dataclass
from typing import List, Optional

# ---------- infra ----------
@dataclass
class Diag:
    code: str        # ELEX*, ESYN*, ESEM*
    severity: str    # "error" | "warning"
    msg: str
    line: int
    col: int
    length: int = 1
    note: Optional[str] = None

def caret(source: str, line: int, col: int, length: int = 1) -> str:
    lines = source.splitlines()
    if line < 1 or line > len(lines): return ""
    s = lines[line-1]
    mark = " "*(max(0,col-1)) + "^" + "~"*max(0,length-1)
    return f"{s}\n{mark}"

class Debug:
    def __init__(self, enabled=False): self.enabled = enabled; self.depth=0
    def log(self, msg):
        if self.enabled: print("  "*self.depth + msg)
    def push(self, title):
        self.log(f"[{title}]"); self.depth += 1
    def pop(self):
        self.depth -= 1; self.log("[/]")

# ---------- analisador simplificado ----------
def analyze(source: str, debug: bool = False) -> List[Diag]:
    dbg = Debug(debug)
    diags: List[Diag] = []
    sym = {}  # nome -> tipo ('int'|'string')
    # pilha de funções: dict(ret, has_return, start_line, brace)
    fstack: List[dict] = []

    lines = source.splitlines()
    for i, line in enumerate(lines, 1):
        dbg.push(f"linha {i}")
        # ---- léxico (simples) ----
        if line.count('"') % 2 == 1:
            c = line.find('"') + 1
            diags.append(Diag("ELEX1","error","string não fechada", i, c))
            dbg.log("ELEX1: string não fechada")
        if '@' in line:
            c = line.index('@') + 1
            diags.append(Diag("ELEX2","error","caractere inválido '@'", i, c))
            dbg.log("ELEX2: caractere inválido '@'")

        # ---- sintaxe ----
        # if (a > ) {...}
        if re.search(r'if\s*\([^)]*\)\s*\{', line):
            if re.search(r'>\s*\)', line):
                c = line.index(')')  # coluna aproximada do ')'
                diags.append(Diag("ESYN1","error","esperado expressão antes de ')'", i, c, note="faltou operando à direita de '>'"))
                dbg.log("ESYN1: esperado expressão antes de ')'")

        # atribuição vazia: x = ;
        if re.match(r'\s*\w+\s*=\s*;\s*$', line):
            c = line.index('=')+1
            diags.append(Diag("ESYN2","error","expressão ausente após '='", i, c))
            dbg.log("ESYN2: expressão ausente")

        # ---- semântica: declarações e atribuições ----
        mdecl = re.match(r'\s*(\w+)\s*:\s*(int|string)\s*;', line)
        if mdecl:
            name, ty = mdecl.group(1), mdecl.group(2)
            if name in sym:
                diags.append(Diag("ESEM0","error",f"'{name}' já declarado", i, line.index(name)+1))
                dbg.log(f"ESEM0: redeclaração de {name}")
            else:
                sym[name] = ty
                dbg.log(f"declara {name}: {ty}")

        massign = re.match(r'\s*(\w+)\s*=\s*(.+?);\s*$', line)
        if massign:
            name, expr = massign.group(1), massign.group(2).strip()
            vt = sym.get(name)
            dbg.log(f"assign {name} (tipo var={vt}) expr='{expr}'")
            if vt is None:
                diags.append(Diag("ESEM2","error",f"variável '{name}' não declarada", i, line.index(name)+1))
            else:
                et = "string" if '"' in expr else ("int" if re.fullmatch(r'\d+', expr) else None)
                if et and et != vt:
                    diags.append(Diag("ESEM1","error",f"tipos incompatíveis: '{vt}' ← '{et}'", i, line.index(name)+1))

        # ---- funções: retorno obrigatório/indevido ----
        mfn = re.match(r'\s*fn\s+(\w+)\s*\((.*?)\)\s*->\s*(\w+)\s*\{\s*$', line)
        if mfn:
            ret = mfn.group(3)
            fstack.append({"ret":ret, "has_return":False, "start_line":i, "brace":1})
            dbg.log(f"função inicia (ret={ret})")

        # contagem bracetes dentro da função
        if fstack:
            # returns
            mret = re.match(r'\s*return(?:\s+(.+?))?;\s*$', line)
            if mret:
                expr = (mret.group(1) or "").strip()
                f = fstack[-1]; f["has_return"]=True
                if f["ret"] == "void":
                    if expr:
                        diags.append(Diag("ESEM4","error","return com valor em função void", i, line.index("return")+1))
                        dbg.log("ESEM4: return com valor em void")
                else:
                    if not expr:
                        diags.append(Diag("ESEM5","error",f"return vazio em função que retorna {f['ret']}", i, line.index("return")+1))
                        dbg.log("ESEM5: return vazio")
                    else:
                        et = "string" if '"' in expr else ("int" if re.fullmatch(r'\d+', expr) else None)
                        if et and et != f["ret"]:
                            diags.append(Diag("ESEM6","error",f"retorno incompatível: esperado {f['ret']}, obtido {et}", i, line.index("return")+1))
                            dbg.log("ESEM6: retorno incompatível")
            # atualizar brace
            opened = line.count('{')
            closed = line.count('}')
            fstack[-1]["brace"] += opened - closed
            if fstack[-1]["brace"] == 0:
                f = fstack.pop()
                if f["ret"] != "void" and not f["has_return"]:
                    diags.append(Diag("ESEM3","error",f"função pode sair sem retorno (retorna {f['ret']})", f["start_line"], 1))
                    dbg.log("ESEM3: função sem retorno obrigatório")

        dbg.pop()
    return diags

def render_diags(diags: List[Diag], source: str) -> str:
    out = []
    for d in diags:
        block = f"{d.severity}[{d.code}] {d.msg} @ {d.line}:{d.col}\n" + caret(source, d.line, d.col, d.length)
        if d.note: block += f"\nnota: {d.note}"
        out.append(block)
    return "\n\n".join(out) or "(sem mensagens)"


Célula 2 — Golden tests (+ reexecução com debug nas falhas)

In [11]:
#@title Golden tests com relatório e reexecução das falhas em debug
from typing import Dict, Any, Tuple

TESTS = [
    # 1) Léxico
    {
        "name": "lex/string_nao_fechada",
        "src": 'print("oi',  # aspas ímpar
        "expect": ["ELEX1", "string não fechada"]
    },
    # 2) Sintaxe
    {
        "name": "syn/if_operando_faltando",
        "src": "if (a > ) { b = 1; }",
        "expect": ["ESYN1", "esperado expressão antes de ')'"]
    },
    # 3) Semântica: tipos
    {
        "name": "sem/tipo_incompativel",
        "src": "x: int;\nx = \"a\";",
        "expect": ["ESEM1", "tipos incompatíveis"]
    },
    # 4) Semântica: função sem retorno e return em void
    {
        "name": "sem/retornos",
        "src": """fn f(a:int)->int {
}
fn g()->void {
  return "x";
}""",
        "expect": ["ESEM3", "pode sair sem retorno", "ESEM4", "void"]
    },
    # 5) Semântica: uso sem declaração
    {
        "name": "sem/nao_declarada",
        "src": "y = 42;",
        "expect": ["ESEM2", "não declarada"]
    },
]

def run_test(t: Dict[str,Any], debug=False) -> Tuple[bool,str]:
    diags = analyze(t["src"], debug=debug)
    out = render_diags(diags, t["src"])
    missing = [s for s in t["expect"] if s not in out]
    ok = (len(missing) == 0)
    return ok, (out if debug or not ok else "")

print("== Executando golden tests ==")
failed = []
for t in TESTS:
    ok, detail = run_test(t, debug=False)
    status = "OK" if ok else "FALHOU"
    print(f"- {t['name']}: {status}")
    if not ok:
        print("  (reexecutando com debug)")
        ok2, detail2 = run_test(t, debug=True)
        print(detail2)
        failed.append(t["name"])

print("\nResumo:")
print(f"Total: {len(TESTS)} | Falhas: {len(failed)}")


== Executando golden tests ==
- lex/string_nao_fechada: OK
- syn/if_operando_faltando: OK
- sem/tipo_incompativel: OK
- sem/retornos: FALHOU
  (reexecutando com debug)
[linha 1]
  função inicia (ret=int)
[/]
[linha 2]
[/]
[linha 3]
  função inicia (ret=void)
[/]
[linha 4]
  ESEM4: return com valor em void
[/]
[linha 5]
[/]
error[ESEM4] return com valor em função void @ 4:3
  return "x";
  ^
- sem/nao_declarada: OK

Resumo:
Total: 5 | Falhas: 1


Célula 3 — Delta debugging: minimizando o caso que falhou

In [12]:
#@title Delta debugging simples (redução por linhas) + demonstração
from typing import Callable

def shrink_by_lines(src: str, still_fails: Callable[[str], bool]) -> str:
    """Remove linhas iterativamente mantendo a propriedade de falha."""
    lines = src.splitlines()
    i = 0
    changed = True
    while changed:
        changed = False
        j = 0
        while j < len(lines):
            candidate = "\n".join(lines[:j] + lines[j+1:])
            if candidate and still_fails(candidate):
                lines = candidate.splitlines()
                changed = True
                # não incrementa j: tenta remover outra na mesma posição
            else:
                j += 1
    return "\n".join(lines)

# Demonstração: vamos reduzir o caso "sem/retornos" procurando pela mensagem 'pode sair sem retorno'
target_sub = "pode sair sem retorno"

def predicate(src: str) -> bool:
    out = render_diags(analyze(src, debug=False), src)
    return target_sub in out

orig = next(t["src"] for t in TESTS if t["name"] == "sem/retornos")
print("== Original ==")
print(orig)
print("\n== Diagnóstico original ==")
print(render_diags(analyze(orig, debug=False), orig))

print("\n== Reduzindo (delta debugging) ==")
reduced = shrink_by_lines(orig, predicate)
print(reduced)
print("\n== Diagnóstico após redução ==")
print(render_diags(analyze(reduced, debug=False), reduced))


== Original ==
fn f(a:int)->int {
}
fn g()->void {
  return "x";
}

== Diagnóstico original ==
error[ESEM4] return com valor em função void @ 4:3
  return "x";
  ^

== Reduzindo (delta debugging) ==
fn f(a:int)->int {
}
fn g()->void {
  return "x";
}

== Diagnóstico após redução ==
error[ESEM4] return com valor em função void @ 4:3
  return "x";
  ^
